In [1]:
# ----数据库操作----
import pymysql
import pymssql
import pandas as pd
import psycopg2
import os
import time

/Users/apache/PycharmProjects/shushan-CF/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
# redshift 数据库连接
con1 = psycopg2.connect(
        user='operation',
        password='Operation123',
        host='jiayundatapro.cls0csjdlwvj.us-west-2.redshift.amazonaws.com',
        port=5439,
        database='jiayundata')

In [3]:
# redshift 取数
sql1=('''
    select
    biz_date
    ,ic.front_cate_one
    ,ic.front_cate_two
    ,campaign_type
    ,cn.id
    ,cn.category_english_name
    ,cn.category_chinese_name
    ,sum(spend) as cost
    ,sum(mobile_app_install) as installs
    ,sum(fb_mobile_purchase+fb_pixel_purchase) as purchase
    ,sum(fb_mobile_purchase_value+fb_pixel_purchase_value) as purchaseValiue
    ,sum(impressions) as impression
    ,sum(clicks_all) as clicks 
    ,country_code as country FROM
(select * from marketing.facebook_dpa_elimination where biz_date>='2019-10-11' and biz_date<='2019-10-24') t1
       left join odoo_own.product_template pro on t1.product_id = pro.product_no
       left join odoo_own_public.category_new_product_template_rel re on re.product_template_id = pro.id
       left join odoo_own_public.category_new cn on cn.id = re.category_new_id 
       LEFT JOIN jiayundw_dim.product_basic_info_df as ic ON ic.item_no = t1.product_id
group by 
     biz_date
    ,ic.front_cate_one
    ,ic.front_cate_two
    ,campaign_type
    ,cn.id
    ,cn.category_english_name
    ,cn.category_chinese_name
    ,country
order by biz_date,ic.front_cate_one,ic.front_cate_two,cn.id,campaign_type
 ''')

In [7]:
data1 = pd.read_sql(sql1,con1)
print(data1.head(10))

     biz_date front_cate_one front_cate_two campaign_type          id  \
0  2019-10-11    Automobiles      Car Cares       catalog  60407113.0   
1  2019-10-11    Automobiles      Car Cares       catalog  60407113.0   
2  2019-10-11    Automobiles      Car Cares       install  60407113.0   
3  2019-10-11    Automobiles      Car Cares       catalog  60407115.0   
4  2019-10-11    Automobiles      Car Cares       catalog  60407119.0   
5  2019-10-11    Automobiles      Car Cares       catalog  60407119.0   
6  2019-10-11    Automobiles      Car Cares       catalog  60407119.0   
7  2019-10-11    Automobiles      Car Cares       catalog  60407119.0   
8  2019-10-11    Automobiles      Car Cares       install  60407119.0   
9  2019-10-11    Automobiles      Car Cares       install  60407119.0   

  category_english_name category_chinese_name  cost  installs  purchase  \
0        Polish & Waxes                    车蜡  0.02         0         0   
1        Polish & Waxes                    车蜡 

In [8]:
# sql server 数据库连接
con2 = pymssql.connect("172.16.92.6","sa","yssshushan2008","CFflows",charset="utf8")

In [9]:
# sql server 取数
sql2 = ('''SELECT top 10* from CFflows.dbo.FB推广数据表 WHERE biz_date >='2019-08-29' ORDER BY biz_date''')

In [10]:
data2 = pd.read_sql(sql2,con2)
print(data2.head(10))

    biz_date   front_cate_one          front_cate_two campaign_type        id  \
0 2019-08-29             Home              Home Decor       install  61402006   
1 2019-08-29      Electronics            Data Storage       install  61003076   
2 2019-08-29      Electronics          Camera & Photo       catalog  61004136   
3 2019-08-29      Electronics           Audio & Video       catalog  62001002   
4 2019-08-29  Beauty & Health               Skin Care       install      None   
5 2019-08-29  Beauty & Health           Nails & Tools       install  61907072   
6 2019-08-29  Beauty & Health  Hair Extensions & Wigs       install  62702016   
7 2019-08-29      Automobiles    Interior Accessories       install  60403049   
8 2019-08-29      Automobiles    Interior Accessories       catalog  60402023   
9 2019-08-29      Automobiles         Car Electronics       catalog      None   

            category_english_name category_chinese_name   cost  installs  \
0        Alarm Clock & Desk Cloc